In [1]:
%pdb
%load_ext autoreload
%autoreload 2

Automatic pdb calling has been turned ON


In [ ]:
%run dqn_atari.py --env "SpaceInvaders-v0" -o ./runs --seed 42

/home/tovitu/miniconda3/envs/drl/lib/python3.10/site-packages/gymnasium/envs/registration.py:517: DeprecationWarning: WARN: The environment SpaceInvaders-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


Iteration: 578 Episode reward: 125.0 Episode length: 578 Loss: 0.007577439661540195 Q-values: 0.06125419959425926 Epsilon: 0.99994807
Iteration: 971 Episode reward: 105.0 Episode length: 393 Loss: 0.004977739415079093 Q-values: 0.059640344232320786 Epsilon: 0.9999127
Iteration: 1794 Episode reward: 215.0 Episode length: 823 Loss: 0.00672402888307861 Q-values: 0.062488406896591187 Epsilon: 0.99983863
Iteration: 2391 Episode reward: 110.0 Episode length: 597 Loss: 0.0067552758113953015 Q-values: 0.06220201030373573 Epsilon: 0.9997849
Iteration: 3213 Episode reward: 210.0 Episode length: 822 Loss: 0.0068200801195822894 Q-values: 0.06299585103988647 Epsilon: 0.99971092
Iteration: 3987 Episode reward: 335.0 Episode length: 774 Loss: 0.006263303123223004 Q-values: 0.06106312945485115 Epsilon: 0.99964126
Iteration: 4593 Episode reward: 130.0 Episode length: 606 Loss: 0.006074571597573116 Q-values: 0.06290063261985779 Epsilon: 0.99958672
Iteration: 5419 Episode reward: 200.0 Episode length: 82